## **Web Scraping de TUCARRO.COM**

### **1. Importamos las respectivas librerías**

In [76]:
from bs4 import BeautifulSoup
import requests
import pandas as pd
import re

### **2. Hacemos el respectivo requerimiento a la página**

In [77]:
html_text = requests.get('https://carros.tucarro.com.co/')
print(html_text.status_code)

200


In [78]:
conexion_url = html_text.url
print(conexion_url)

https://carros.tucarro.com.co/


### **3. Seleccionamos la grilla específica de donde vamos a sacar la información**

In [79]:
soup = BeautifulSoup(html_text.text, 'lxml')
cars_grill = soup.find('ol', class_ = 'ui-search-layout ui-search-layout--grid')
cars = cars_grill.find_all('li', class_ = 'ui-search-layout__item')

### **4. Extraemols la información correspondiente**

Necesitamos guardar la información que vamos a extraer de la página web. Para esta aplicación, vamos a usar la memoria del sistema para almancenar los datos a través de un dataframe

**Observación:** una aplicación real requiere montar un back-end donde la información persista una vez es descargada

In [80]:
df = pd.DataFrame()

#### 4.1 Nombre del carro

In [81]:
#Creamos una lista vacio donde vamos a almacenar la informacion
car_name_list = []

for car in cars:
    car_name = car.find('img', alt = True)
    
    #Construimos las expresión regular para identificar el nombre del carro
    pattern = r'<img[^>]*alt="([^"]*)"'
    car_name_adjusted = re.findall(pattern, str(car_name))
    car_name_list.append(car_name_adjusted)

df = pd.DataFrame(car_name_list, columns=['Nombre_del_vehiculo'])
print(df.head(5))

          Nombre_del_vehiculo
0     Jeep Renegade 1.8 Sport
1    Kia Sportage 2.0 Gt Line
2      Chevrolet Aveo 1.6 Gti
3         Renault Trafic 1.6l
4  Mazda B2200 2.2 B22cs9 4x2


#### 4.2 Precio del carro

In [82]:
#Creamos una lista vacio donde vamos a almacenar la informacion
price_car_list = []

for car in cars:
    price_car = car.find('span', class_ = 'andes-money-amount ui-search-price__part ui-search-price__part--medium andes-money-amount--cents-superscript')

    #Construimos las expresión regular para identificar el precio
    pattern = r'aria-label="(\d+)'

    price_car_adjusted = re.findall(pattern, str(price_car))
    price_car_adjusted = int(price_car_adjusted[0])
    price_car_list.append(price_car_adjusted)

df['Precio_del_vehiculo'] = price_car_list
print(df.head(5))

          Nombre_del_vehiculo  Precio_del_vehiculo
0     Jeep Renegade 1.8 Sport             74000000
1    Kia Sportage 2.0 Gt Line            129000000
2      Chevrolet Aveo 1.6 Gti             24000000
3         Renault Trafic 1.6l             92000000
4  Mazda B2200 2.2 B22cs9 4x2             35000000


#### 4.3 Model y Kilometraje del carro

In [83]:
#Creamos una lista vacio donde vamos a almacenar la informacion
model_car_list = []
km_car_list = []

for car in cars:
    # Extraemos el modelo del carro
    model_car = int(car.find_all('li', class_ = 'ui-search-card-attributes__attribute')[0].text)
    model_car_list.append(model_car)

    # Extraemos el kilometraje del carro
    km_car = car.find_all('li', class_ = 'ui-search-card-attributes__attribute')[1].text
    km_car = int(km_car.replace(" Km","").replace(".",""))
    km_car_list.append(km_car)

df['Modelo_del_vehiculo'] = model_car_list
df['Kilometraje_del_vehiculo'] = km_car_list
print(df.head(5))

          Nombre_del_vehiculo  Precio_del_vehiculo  Modelo_del_vehiculo  \
0     Jeep Renegade 1.8 Sport             74000000                 2020   
1    Kia Sportage 2.0 Gt Line            129000000                 2022   
2      Chevrolet Aveo 1.6 Gti             24000000                 2009   
3         Renault Trafic 1.6l             92000000                 2021   
4  Mazda B2200 2.2 B22cs9 4x2             35000000                 2003   

   Kilometraje_del_vehiculo  
0                     50143  
1                     42890  
2                     97688  
3                    107933  
4                    350000  


#### 4.4 Ubicación del carro

In [84]:
#Creamos una lista vacio donde vamos a almacenar la informacion
location_car_list = []

for car in cars:
    location_car = car.find('span', class_ = 'ui-search-item__group__element ui-search-item__location').text
    location_car_list.append(location_car)

df['Ubicacion_del_vehiculo'] = location_car_list
print(df.head(5))

          Nombre_del_vehiculo  Precio_del_vehiculo  Modelo_del_vehiculo  \
0     Jeep Renegade 1.8 Sport             74000000                 2020   
1    Kia Sportage 2.0 Gt Line            129000000                 2022   
2      Chevrolet Aveo 1.6 Gti             24000000                 2009   
3         Renault Trafic 1.6l             92000000                 2021   
4  Mazda B2200 2.2 B22cs9 4x2             35000000                 2003   

   Kilometraje_del_vehiculo            Ubicacion_del_vehiculo  
0                     50143             Usaquén - Bogotá D.C.  
1                     42890             Kennedy - Bogotá D.C.  
2                     97688             Kennedy - Bogotá D.C.  
3                    107933  Rafael Uribe Uribe - Bogotá D.C.  
4                    350000         Teusaquillo - Bogotá D.C.  


In [85]:
#Creamos una lista vacio donde vamos a almacenar la informacion
links_car_list = []

for car in cars:
    link_car = car.find('a', class_ = 'ui-search-link')['href']
    links_car_list.append(link_car)

df['link'] = links_car_list
print(df.head(5))

          Nombre_del_vehiculo  Precio_del_vehiculo  Modelo_del_vehiculo  \
0     Jeep Renegade 1.8 Sport             74000000                 2020   
1    Kia Sportage 2.0 Gt Line            129000000                 2022   
2      Chevrolet Aveo 1.6 Gti             24000000                 2009   
3         Renault Trafic 1.6l             92000000                 2021   
4  Mazda B2200 2.2 B22cs9 4x2             35000000                 2003   

   Kilometraje_del_vehiculo            Ubicacion_del_vehiculo  \
0                     50143             Usaquén - Bogotá D.C.   
1                     42890             Kennedy - Bogotá D.C.   
2                     97688             Kennedy - Bogotá D.C.   
3                    107933  Rafael Uribe Uribe - Bogotá D.C.   
4                    350000         Teusaquillo - Bogotá D.C.   

                                                link  
0  https://articulo.tucarro.com.co/MCO-1451518341...  
1  https://articulo.tucarro.com.co/MCO-145151820

#### 4.5 Marca del vehículo

Auno no sabemos la marca del vehículo, necesitamos extraerla de la columna *'Nombre_del_vehiculo'*

In [86]:
df['Marca_del_vehiculo'] = df['Nombre_del_vehiculo'].str.split(" ").str[0]
print(df.head(5))

          Nombre_del_vehiculo  Precio_del_vehiculo  Modelo_del_vehiculo  \
0     Jeep Renegade 1.8 Sport             74000000                 2020   
1    Kia Sportage 2.0 Gt Line            129000000                 2022   
2      Chevrolet Aveo 1.6 Gti             24000000                 2009   
3         Renault Trafic 1.6l             92000000                 2021   
4  Mazda B2200 2.2 B22cs9 4x2             35000000                 2003   

   Kilometraje_del_vehiculo            Ubicacion_del_vehiculo  \
0                     50143             Usaquén - Bogotá D.C.   
1                     42890             Kennedy - Bogotá D.C.   
2                     97688             Kennedy - Bogotá D.C.   
3                    107933  Rafael Uribe Uribe - Bogotá D.C.   
4                    350000         Teusaquillo - Bogotá D.C.   

                                                link Marca_del_vehiculo  
0  https://articulo.tucarro.com.co/MCO-1451518341...               Jeep  
1  https:/

#### 4.6 Timestand de la operación y metadata

Es importante que siempre haya un registro histórico de las operaciones para efectos de trazabilidad. Otra metadata que podríamos agregar es la página de la que se está extrayendo la información

In [87]:
df['Registro_pagina_principal'] = pd.Timestamp.now()
df['origen_general'] = conexion_url
df

,Nombre_del_vehiculo,Precio_del_vehiculo,Modelo_del_vehiculo,Kilometraje_del_vehiculo,Ubicacion_del_vehiculo,link,Marca_del_vehiculo,Registro_pagina_principal,origen_general
0,Jeep Renegade 1.8 Sport,74000000,2020,50143,Usaquén - Bogotá D.C.,https://articulo.tucarro.com.co/MCO-1451518341...,Jeep,2024-07-12 17:14:42.512251,https://carros.tucarro.com.co/
1,Kia Sportage 2.0 Gt Line,129000000,2022,42890,Kennedy - Bogotá D.C.,https://articulo.tucarro.com.co/MCO-1451518201...,Kia,2024-07-12 17:14:42.512251,https://carros.tucarro.com.co/
2,Chevrolet Aveo 1.6 Gti,24000000,2009,97688,Kennedy - Bogotá D.C.,https://articulo.tucarro.com.co/MCO-2527033022...,Chevrolet,2024-07-12 17:14:42.512251,https://carros.tucarro.com.co/
3,Renault Trafic 1.6l,92000000,2021,107933,Rafael Uribe Uribe - Bogotá D.C.,https://articulo.tucarro.com.co/MCO-2526980992...,Renault,2024-07-12 17:14:42.512251,https://carros.tucarro.com.co/
4,Mazda B2200 2.2 B22cs9 4x2,35000000,2003,350000,Teusaquillo - Bogotá D.C.,https://articulo.tucarro.com.co/MCO-1451430511...,Mazda,2024-07-12 17:14:42.512251,https://carros.tucarro.com.co/
5,Chevrolet Captiva 2.4 Sport,28800000,2011,110000,Usaquén - Bogotá D.C.,https://articulo.tucarro.com.co/MCO-2526954900...,Chevrolet,2024-07-12 17:14:42.512251,https://carros.tucarro.com.co/
6,Volkswagen Amarok 2.0 Trendline Diesel,73300000,2013,139000,Rionegro - Antioquia,https://articulo.tucarro.com.co/MCO-2526941198...,Volkswagen,2024-07-12 17:14:42.512251,https://carros.tucarro.com.co/
7,Renault Logan 1.6 Authentique,33800000,2017,87000,Usaquén - Bogotá D.C.,https://articulo.tucarro.com.co/MCO-2526991636...,Renault,2024-07-12 17:14:42.512251,https://carros.tucarro.com.co/
8,Nissan Versa 1.6 Sense,46500000,2018,52000,Kennedy - Bogotá D.C.,https://articulo.tucarro.com.co/MCO-2526962880...,Nissan,2024-07-12 17:14:42.512251,https://carros.tucarro.com.co/
9,Suzuki Swift 1.2 Hybrid,66000000,2023,16700,Cota - Cundinamarca,https://articulo.tucarro.com.co/MCO-2526923300...,Suzuki,2024-07-12 17:14:42.512251,https://carros.tucarro.com.co/


En este punto podemos usar las propiedades de tablas SQL para vincular la información general que se está extrayendo en este punto, con la que vamos a extraer posteriormente para cada uno de los items.

Para tal efecto, necesitamos tener una llave que una ambas tablas, y en este caso, la llave es el identficador de cada vehículo que aparece en la url y que inicia con MCO.

In [88]:
def extract_mco_code(url):
    import re
    pattern = r'MCO-\d+'
    match = re.search(pattern, url)
    if match:
        return match.group()
    else:
        return None

df['Key'] = df['link'].apply(extract_mco_code)
df

,Nombre_del_vehiculo,Precio_del_vehiculo,Modelo_del_vehiculo,Kilometraje_del_vehiculo,Ubicacion_del_vehiculo,link,Marca_del_vehiculo,Registro_pagina_principal,origen_general,Key
0,Jeep Renegade 1.8 Sport,74000000,2020,50143,Usaquén - Bogotá D.C.,https://articulo.tucarro.com.co/MCO-1451518341...,Jeep,2024-07-12 17:14:42.512251,https://carros.tucarro.com.co/,MCO-1451518341
1,Kia Sportage 2.0 Gt Line,129000000,2022,42890,Kennedy - Bogotá D.C.,https://articulo.tucarro.com.co/MCO-1451518201...,Kia,2024-07-12 17:14:42.512251,https://carros.tucarro.com.co/,MCO-1451518201
2,Chevrolet Aveo 1.6 Gti,24000000,2009,97688,Kennedy - Bogotá D.C.,https://articulo.tucarro.com.co/MCO-2527033022...,Chevrolet,2024-07-12 17:14:42.512251,https://carros.tucarro.com.co/,MCO-2527033022
3,Renault Trafic 1.6l,92000000,2021,107933,Rafael Uribe Uribe - Bogotá D.C.,https://articulo.tucarro.com.co/MCO-2526980992...,Renault,2024-07-12 17:14:42.512251,https://carros.tucarro.com.co/,MCO-2526980992
4,Mazda B2200 2.2 B22cs9 4x2,35000000,2003,350000,Teusaquillo - Bogotá D.C.,https://articulo.tucarro.com.co/MCO-1451430511...,Mazda,2024-07-12 17:14:42.512251,https://carros.tucarro.com.co/,MCO-1451430511
5,Chevrolet Captiva 2.4 Sport,28800000,2011,110000,Usaquén - Bogotá D.C.,https://articulo.tucarro.com.co/MCO-2526954900...,Chevrolet,2024-07-12 17:14:42.512251,https://carros.tucarro.com.co/,MCO-2526954900
6,Volkswagen Amarok 2.0 Trendline Diesel,73300000,2013,139000,Rionegro - Antioquia,https://articulo.tucarro.com.co/MCO-2526941198...,Volkswagen,2024-07-12 17:14:42.512251,https://carros.tucarro.com.co/,MCO-2526941198
7,Renault Logan 1.6 Authentique,33800000,2017,87000,Usaquén - Bogotá D.C.,https://articulo.tucarro.com.co/MCO-2526991636...,Renault,2024-07-12 17:14:42.512251,https://carros.tucarro.com.co/,MCO-2526991636
8,Nissan Versa 1.6 Sense,46500000,2018,52000,Kennedy - Bogotá D.C.,https://articulo.tucarro.com.co/MCO-2526962880...,Nissan,2024-07-12 17:14:42.512251,https://carros.tucarro.com.co/,MCO-2526962880
9,Suzuki Swift 1.2 Hybrid,66000000,2023,16700,Cota - Cundinamarca,https://articulo.tucarro.com.co/MCO-2526923300...,Suzuki,2024-07-12 17:14:42.512251,https://carros.tucarro.com.co/,MCO-2526923300


### **5. Identificamos las páginas que vamos a consultar**

Para este ejercicio vamos a consultar únicamente las primeras 10 páginas disponibles

Vamos primero a declarar un diccionario que nos ayude a documentar las páginas en las que nos vamos a mover, la inicial termina con sufijo **"/_NoIndex_True"**

In [89]:
paginas = {}

Podemos observar que cada página tiene 48 elementos, y que la segunda página arranca en *"49 - _Desde_49_NoIndex_True"*, la tercera en *97 - _Desde_97_NoIndex_True*, es decir con 48 elementos de diferencia... **"hay está el patrón"**

In [90]:
for i in range(1,11):
    if i == 1:
        paginas[str(i)] = f'_NoIndex_True'
    else:
        paginas[str(i)] = f'_Desde_{48*(i-1)+1}_NoIndex_True'
    print(paginas)

{'1': '_NoIndex_True'}
{'1': '_NoIndex_True', '2': '_Desde_49_NoIndex_True'}
{'1': '_NoIndex_True', '2': '_Desde_49_NoIndex_True', '3': '_Desde_97_NoIndex_True'}
{'1': '_NoIndex_True', '2': '_Desde_49_NoIndex_True', '3': '_Desde_97_NoIndex_True', '4': '_Desde_145_NoIndex_True'}
{'1': '_NoIndex_True', '2': '_Desde_49_NoIndex_True', '3': '_Desde_97_NoIndex_True', '4': '_Desde_145_NoIndex_True', '5': '_Desde_193_NoIndex_True'}
{'1': '_NoIndex_True', '2': '_Desde_49_NoIndex_True', '3': '_Desde_97_NoIndex_True', '4': '_Desde_145_NoIndex_True', '5': '_Desde_193_NoIndex_True', '6': '_Desde_241_NoIndex_True'}
{'1': '_NoIndex_True', '2': '_Desde_49_NoIndex_True', '3': '_Desde_97_NoIndex_True', '4': '_Desde_145_NoIndex_True', '5': '_Desde_193_NoIndex_True', '6': '_Desde_241_NoIndex_True', '7': '_Desde_289_NoIndex_True'}
{'1': '_NoIndex_True', '2': '_Desde_49_NoIndex_True', '3': '_Desde_97_NoIndex_True', '4': '_Desde_145_NoIndex_True', '5': '_Desde_193_NoIndex_True', '6': '_Desde_241_NoIndex_True